## Preambule

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Input

In [3]:
rules_b = ['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']
rules_a = ['GF', 'PC', 'PCC', 'AP', 'GDR']

## Read data

In [4]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
#xr_2030 = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_alloc_2030.nc")

In [5]:
df_eng = pd.read_csv('X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv')
df_eng = df_eng[df_eng.Variable == 'Emissions|CO2']
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = 'ARG'
regions_df[regions_df == "Canada"] = 'CAN'
regions_df[regions_df == "Commonwealth of Australia"] = 'AUS'
regions_df[regions_df == "Federative Republic of Brazil"] = 'BRA'
regions_df[regions_df == "People's Repulic of China"] = 'CHN'
regions_df[regions_df == "European Union (28 member countries)"] = 'EU'
regions_df[regions_df == "Republic of India"] = 'IND'
regions_df[regions_df == "Republic of Indonesia"] = 'IDN'
regions_df[regions_df == "State of Japan"] = 'JPN'
regions_df[regions_df == "Russian Federation"] = 'RUS'
regions_df[regions_df == "Kingdom of Saudi Arabia"] = 'SAU'
regions_df[regions_df == "Republic of South Africa"] = 'ZAF'
regions_df[regions_df == "Republic of Korea (South Korea)"] = 'KOR'
regions_df[regions_df == "United Mexican States"] = 'MEX'
regions_df[regions_df == "Republic of Turkey"] = 'TUR'
regions_df[regions_df == "United States of America"] = 'USA'
regions_df[regions_df == "Viet Nam "] = 'VNM'
df_eng.Region = regions_df

In [6]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[['Model', 'Scenario', 'Region']+list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = 'NDC'
scen[scen == curpol] = 'CurPol'
scen[scen == nz] = 'NetZero'
reg = np.array(df_eng_ref.Region)
reg[reg == 'World'] = 'WORLD'
df_eng_ref['Scenario'] = scen
df_eng_ref['Region'] = reg
dummy = df_eng_ref.melt(id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time = np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

## Total budgets - globe

In [7]:
fig = make_subplots(rows=1, cols=1,
                    horizontal_spacing = 0.07, vertical_spacing=0.06)

for T_i, T in enumerate([1.4, 1.5, 1.7, 2.0, 2.3]):
    vals = np.array(xr_dataread.Budget.sel(Temperature=T)).flatten()
    # fig.add_traces(go.Violin(x=[str(T)+"&deg;C<br>(50-83%<br>probability)"]*len(vals),
    #                         y=vals,
    #                         showlegend=False,
    #                         points=False,
    #                         opacity=0.3,
    #                         line_color=['forestgreen', 'lightgreen', 'goldenrod', 'tomato', 'darkred'][T_i]), 1, 1
    #                 )
    # vals = np.array(xr_dataread.Budget.sel(Temperature=T)).flatten()
    # fig.add_traces(go.Violin(x=[str(T)+"&deg;C<br>(50-83%<br>probability)"]*len(vals),
    #                         y=vals,
    #                         showlegend=False,
    #                         points=False,
    #                         opacity=0.3,
    #                         line_color=['forestgreen', 'lightgreen', 'goldenrod', 'tomato', 'darkred'][T_i]), 1, 1
    #                 )
    vals = np.array(xr_dataread.Budget.sel(Temperature=T)).flatten()
    fig.add_traces(go.Violin(x=[str(T)+"&deg;C<br>(50-83%<br>probability)"]*len(vals),
                            y=vals,
                            showlegend=False,
                            points=False,
                            line_color=['forestgreen', 'lightgreen', 'goldenrod', 'tomato', 'darkred'][T_i]), 1, 1
                    )
for i in range(2):
    temp = [1.5, 2.0][i]
    risk = [0.5, 0.33][i]
    val = float(xr_dataread.Budget.sel(Temperature=temp, Risk=risk))
    fig.add_traces(go.Scatter(x=[str(temp)+"&deg;C<br>(50-83%<br>probability)", str(temp)+"&deg;C<br>(50-83%<br>probability)"],
                            y=[val, val],
                            line={'color': ['lightgreen', 'tomato'][i], 'width': 0.75, 'dash':'dot'},
                            marker={'size': 15},
                            mode='lines+markers',
                            showlegend=False), 1, 1
            )
    fig.add_traces(go.Scatter(x=[str(temp)+"&deg;C<br>(50-83%<br>probability)"],
                            y=[val],
                            text="<b>"+str(int(np.round(val, 0)))+'</b> Gt CO<sub>2</sub>e',
                            line={'color': ['lightgreen', 'tomato'][i], 'width': 0.5},
                            textfont={'color': 'black', 'size': 30},
                            textposition='top center',
                            mode='text',
                            showlegend=False), 1, 1
            )

fig.update_layout(height=700, width=1500, template='simple_white')
fig.update_layout(
    yaxis=dict(tickfont=dict(size=25), title='Global CO<sub>2</sub> budget (Gt CO<sub>2</sub>)<br>between 2021-2100', titlefont=dict(size=25)),
    xaxis=dict(tickfont=dict(size=25))
)
fig.update_yaxes(range=(0, 4000))
fig.update_layout(legend=dict(x=1, y=1, xanchor='right', yanchor='top'))
fig.show()
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/GlobalBudgets.png", scale=3)